## Comparison of ReAct and Plan-execute agents

In [1]:
SQL_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movies_meta.db'

PLOT_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv'
REVIEW_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv'
POSTER_PATH = '/Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv'

In [2]:
from src.langchain.chains.movie_rag import MovieRAGChain
from src. langchain.loaders import MoviePosterDocumentLoader
from src.retrievers.visual_retriever import VisualRetriever
from src.langchain.prompts import ZERO_SHOT_QA_PROMPT

In [3]:
poster_loader = MoviePosterDocumentLoader(POSTER_PATH, max_movies=1000)   # small number since model is local
visual_docs = poster_loader.load()
visual_retriever = VisualRetriever(model_name="ViT-B/32", use_text_fusion=True, alpha=0.8)
visual_retriever.add_documents(visual_docs)

MAX_MOVIES = 500  # Limit for faster demos

full_chain = MovieRAGChain(
    plots_path=PLOT_PATH,
    reviews_path=REVIEW_PATH,
    max_movies=MAX_MOVIES,
    use_custom_retriever=True,
    use_custom_chunk=True,
    custom_prompt=ZERO_SHOT_QA_PROMPT,
    k=5,
    use_hyde=True,
    hyde_model="gpt-4o-mini",
    use_reranking=True,
    reranker_cfg={'type':'llm'},
    initial_k=20, 
)
full_chain.build()

Loading posters from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_posters.csv...
Created 947 poster docs.
  ✓ 947 poster documents
Loading CLIP model: ViT-B/32
✓ CLIP loaded on cpu
✓ VisualRetriever (text_fusion=True, method=weight_average)
  Alpha: 0.8 (image=0.8, text=0.2)
Encoding 947 posters with CLIP...
Encoding 947 text descriptions with CLIP...
✓ Fused embeddings using weight_average method (dim=512)
✓ Added 947 posters to index
✓ MovieRAGChain initialized
  Retriever type: custom + reranking + HyDE
  LLM: gpt-4o-mini

Building RAG Pipeline

1. Loading documents...
Limiting to 500 movies
Loading plots from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/movie_plots.csv...
Created 383 plot docs.
  ✓ 383 plot documents
Loading reviews from /Users/saghar/Desktop/movie-rag/datasets/rotten-tomatoes-reviews/prep/reviews_w_movies_full.csv...
Created 500 review docs.
  ✓ 500 review documents
✓ Total: 883 reviews and plots documents

2

In [4]:
from src.agents.react import ReactAgent
from src.agents.plan_execute import PlanExecuteAgent

In [4]:
agent_react = ReactAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH)
agent_plan = PlanExecuteAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH)

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!
Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!


In [5]:
query = "What is Inception about?"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

What is Inception about?
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_N8QM8xzipTDfl9qV45ejbTEV)
 Call ID: call_N8QM8xzipTDfl9qV45ejbTEV
  Args:
    query: Inception
================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include details about the movie "Inception." If you have specific questions about "Inception," please provide more context or details.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_YeTMRZ9bnN7okRa6Vcu4cznf)
 Call ID: call_YeTMRZ9bnN7okRa6Vcu4cznf
  Args:
    query: What is the movie Inception about?
================================= Tool Message =================================
Name: search_movies_by_content

I don't have information about 

In [6]:
query = "drama about love and lust with engaging red poster"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

drama about love and lust with engaging red poster
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content_and_visual (call_0i6r0T83OsFXZw5ZGYpYU4bn)
 Call ID: call_0i6r0T83OsFXZw5ZGYpYU4bn
  Args:
    query: drama about love and lust with engaging red poster
================================= Tool Message =================================
Name: search_movies_by_content_and_visual

Based on your request for a drama about love and lust with an engaging red poster, the film "Fifty Shades of Grey" (2015) stands out as a strong candidate. 

**Textual Alignment:**
"Fifty Shades of Grey" explores themes of love and lust through its narrative centered on a complex romantic relationship that delves into emotional and physical intimacy. The film is well-known for its portrayal of BDSM and the dynamics of desire, making it a fitting choice for your crit

In [13]:
query = "comedy movies featuring tom hanks with good critic reviews"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

comedy movies featuring tom hanks with good critic reviews
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_5VqgPxf7QxdbJJ3k3amEzjz2)
 Call ID: call_5VqgPxf7QxdbJJ3k3amEzjz2
  Args:
    query: comedy movies featuring Tom Hanks with good critic reviews
================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include any comedy movies featuring Tom Hanks with good critic reviews. It primarily focuses on "Apollo 13," which is an adventure film, and "Risky Business," which is a comedy but does not feature Tom Hanks. Therefore, I cannot provide a specific answer based on the available data.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_Mjb6TWp4qjrjLQHSBD121T0Y)
 Call

In [8]:
query = "Find dark sci-fi movies rated above 8 and explain their common themes"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Find dark sci-fi movies rated above 8 and explain their common themes
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_YfSzIgeQ26ALXtCj5w8M4nvW)
 Call ID: call_YfSzIgeQ26ALXtCj5w8M4nvW
  Args:
    question: dark sci-fi movies rated above 8
  search_movies_by_content (call_kal22bWS5anYGnEhPUCx3bQl)
 Call ID: call_kal22bWS5anYGnEhPUCx3bQl
  Args:
    query: common themes in dark sci-fi movies
================================= Tool Message =================================
Name: search_movies_by_content

Common themes in dark sci-fi movies often include the exploration of humanity's relationship with technology, existential dread, and the fear of the unknown. For instance, "Alien" presents space as a frightening trap and blurs the line between man and machine, creating a sense of terror and isolation. Similarly, "Transcendence" add

In [9]:
query = "I loved Inception, Matrix, and Interstellar. Recommend 3 movies and explain why each fits my taste."

for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

I loved Inception, Matrix, and Interstellar. Recommend 3 movies and explain why each fits my taste.
================================== Ai Message ==================================
Tool Calls:
  recommend_by_similar_taste (call_qu2fkI3A6rNZdfxnPtkGyzcF)
 Call ID: call_qu2fkI3A6rNZdfxnPtkGyzcF
  Args:
    movie_titles: Inception, Matrix, Interstellar
    num_recommendations: 3
================================= Tool Message =================================
Name: recommend_by_similar_taste

{
  "warnings": "Warning: Multiple close matches found for Inception, it might also not be available in dataset. Options: ['inception (2010)', 'deception (2008)', 'reindeer games (deception) (2000)']. Warning: Multiple close matches found for Matrix, it might also not be available in dataset. Options: ['the matrix (1999)', 'the matrix reloaded (2003)', 'the matrix revolutions (2003)']. ",
  "input_movies": [
    "Interst

In [10]:
query = "Compare Wes Anderson’s films to other colorful comedies. What defines his visual style and how do their audience ratings differ?"

for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Compare Wes Anderson’s films to other colorful comedies. What defines his visual style and how do their audience ratings differ?
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content_and_visual (call_5UxQPgkGIPO4UqNUAfEIU8mz)
 Call ID: call_5UxQPgkGIPO4UqNUAfEIU8mz
  Args:
    query: Wes Anderson films visual style and colorful comedies
  search_movies_by_metadata (call_2MoNvLBLvnbTEfWVbPhfriOL)
 Call ID: call_2MoNvLBLvnbTEfWVbPhfriOL
  Args:
    question: Compare audience ratings of Wes Anderson films to other colorful comedies
================================= Tool Message =================================
Name: search_movies_by_metadata

          film_category  average_audience_rating
Other Colorful Comedies                60.293601
           Wes Anderson                84.888889
================================== Ai Message ==========

In [11]:
query = "Between Christopher Nolan and Denis Villeneuve, whose recent sci-fi films are rated higher, and what themes do they share?"

for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Between Christopher Nolan and Denis Villeneuve, whose recent sci-fi films are rated higher, and what themes do they share?
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_dwyfG7tBmkm9SgklrjL3qjeZ)
 Call ID: call_dwyfG7tBmkm9SgklrjL3qjeZ
  Args:
    question: Recent sci-fi films by Christopher Nolan and Denis Villeneuve with ratings comparison
  search_movies_by_content (call_AlBcGgY83c6kvXHYZnYM3NOj)
 Call ID: call_AlBcGgY83c6kvXHYZnYM3NOj
  Args:
    query: themes in recent sci-fi films by Christopher Nolan and Denis Villeneuve
================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include specific themes from recent sci-fi films by Christopher Nolan and Denis Villeneuve. It only discusses the film "Transcendence," directed by Wally Pfi

In [12]:
query = "Analyze how critic sentiment towards superhero movies has changed from the 2000s to the 2020s."

for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

Analyze how critic sentiment towards superhero movies has changed from the 2000s to the 2020s.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_AVBOA8ZGGT4BHbO3g4tZLMXr)
 Call ID: call_AVBOA8ZGGT4BHbO3g4tZLMXr
  Args:
    question: What are the average critic ratings for superhero movies from the 2000s?
  search_movies_by_metadata (call_EQDEtK5JCx1svkNKCw1TTYC3)
 Call ID: call_EQDEtK5JCx1svkNKCw1TTYC3
  Args:
    question: What are the average critic ratings for superhero movies from the 2010s?
  search_movies_by_metadata (call_07nsbjEa3l1VKmaP4nAObzTo)
 Call ID: call_07nsbjEa3l1VKmaP4nAObzTo
  Args:
    question: What are the average critic ratings for superhero movies from the 2020s?
================================= Tool Message =================================
Name: search_movies_by_metadata

average_tomatometer_rating aver

#### Testing and comparing web-search tool for both

In [5]:
agent_react = ReactAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)
agent_plan = PlanExecuteAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!
Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!


In [6]:
query = "When is Severence new season going to be released?"
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

When is Severence new season going to be released?
================================== Ai Message ==================================
Tool Calls:
  search_web_for_movies (call_UdrFP5pq8pajtR45hciZBWPo)
 Call ID: call_UdrFP5pq8pajtR45hciZBWPo
  Args:
    query: Severance new season release date
================================= Tool Message =================================
Name: search_web_for_movies

Result 1: Severance Season 3: Release Date, Cast Changes, and What's ...
*Severance* **is officially coming back for season 3**. Here’s everything we know right now about Apple’s biggest sci-fi hit, from new writers to filming dates and when it might finally return to Apple TV+. ### Severance Season 3 Officially Greenlit by Apple In February, Ben Stiller (the guy who directs and produces the show) told *The Hollywood Reporter* that work on season 3 was “underway,” with a full writers’ room already going strong

In [7]:
query = "What is one of the best rated movies of 2025? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies."
for agent in agent_react, agent_plan:
    _ = agent.query(query, verbose=True)

================================ Human Message =================================

What is one of the best rated movies of 2025? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies.
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_metadata (call_Ejdv8Lm4FmyKCvB91Bsj1KyS)
 Call ID: call_Ejdv8Lm4FmyKCvB91Bsj1KyS
  Args:
    question: Best rated movies of 2025
================================= Tool Message =================================
Name: search_movies_by_metadata

No movies found matching: Best rated movies of 2025. Tried query: SQL: SELECT movie_title, imdb_rating, tomatometer_rating, audience_rating
FROM movies
WHERE release_year = 2025
ORDER BY imdb_rating DESC, tomatometer_rating DESC, audience_rating DESC;
================================== Ai Message ==================================
Tool Calls:
  search_web_fo

##### We can see that plan-execute can fail if we don't allow replanning!

## Plan-execute agent with replanning capability

In [ ]:
from src.agents.adaptive_plan_execute import AdaptivePlanExecuteAgent

In [36]:
agent_plan_ad = AdaptivePlanExecuteAgent(full_chain, visual_retriever, SQL_PATH, REVIEW_PATH, web_search_enabled=True)

Creating tools...
Loaded 6127 critics, 8075 movies, 546252 ratings.
Computed movie similarity matrix.
Building agent graph...
✓ Agent ready!


In [37]:
query = "what is inception about?"
_ = agent_plan_ad.query(query, verbose=True)

================================ Human Message =================================

what is inception about?

Plan created (2 steps):
  1. Use [search_movies_by_content] to get a detailed summary of the plot and themes of "Inception."
  2. Synthesize results to answer the question about what "Inception" is about.

Executing step 1/2: 1. Use [search_movies_by_content] to get a detailed summary of the plot and themes of "Inception."
================================== Ai Message ==================================
Tool Calls:
  search_movies_by_content (call_oXP3QMpcUumzRUJMls8EoFUc)
 Call ID: call_oXP3QMpcUumzRUJMls8EoFUc
  Args:
    query: Inception
================================= Tool Message =================================
Name: search_movies_by_content

The provided information does not include details about the movie "Inception." If you have specific questions about "Inception," please provide more context or details.
 Step failed: the provided information does not include details 

In [44]:
query = "What is one of the most well received horror movies of 2025 in terms of imdb? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies."
_ = agent_plan_ad.query(query, verbose=True)

================================ Human Message =================================

What is one of the most well received horror movies of 2025 in terms of imdb? Find the main actor/actress's name. See what type of movies they have acted in, and what other movies you might like if you're a fan of this actor's movies.

Plan created (5 steps):
  1. Use **search_web_for_movies** to find the most well-received horror movies of 2025 in terms of IMDb ratings. This will provide the title of a highly rated horror movie released in that year.
  2. Use **search_movies_by_metadata** to find the main actor/actress's name for the identified horror movie. This will give us the key cast member associated with the film.
  3. Use **search_movies_by_content** to find out what type of movies the main actor/actress has acted in. This will help us understand their filmography and the genres they are associated with.
  4. Use **recommend_by_similar_taste** to find other movies that you might like if you're a 

In [42]:
query = "I love normal people. Can you tell me who the main actor was and what type of movies he's acted in? If I like that movie what else might I be interested in?"
_ = agent_plan_ad.query(query, verbose=True)

================================ Human Message =================================

I love normal people. Can you tell me who the main actor was and what type of movies he's acted in? If I like that movie what else might I be interested in?

Plan created (3 steps):
  1. Use **search_movies_by_content** to find out who the main actor in "Normal People" is and gather information about the types of movies he has acted in.
  2. Use **recommend_by_similar_taste** to find movies that are similar to "Normal People" based on critic rating patterns, which will help identify what else the user might be interested in.
  3. Synthesize results to provide the user with the main actor's name, the types of movies he has acted in, and a list of recommended movies based on their interest in "Normal People."

Executing step 1/3: 1. Use **search_movies_by_content** to find out who the main actor in "Normal People" is and gather information about the types of movies he has acted in.
=========================